# 第4章 顧客の行動を予測する10本ノック (31~40)

In [1]:
# 下準備として，データのあるディレクトリに移動しておく
import os

DATA_ROOT_DIR = "./sample/4章/"
os.chdir(DATA_ROOT_DIR)

## ノック31 データを読み込んで確認しよう
例のごとく、まずはデータを読み込む

In [2]:
import pandas as pd
use_log = pd.read_csv("use_log.csv")
use_log.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [3]:
customer = pd.read_csv("customer_join.csv")
customer.isnull().sum()

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64